In [6]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions, ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
# from selenium.webdriver.support.expected_conditions import staleness_of
import pandas as pd 
import time
import re
import os
import requests

In [19]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(r"D:\GS_WebScarping\project_12_1link\chromedriver.exe")
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)
    
    # driver.maximize_window()
    # ====================================================================================================================
    # get the username tab
    try:
        provided_u_name = "EG78TP9"
        user_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'.//input[@placeholder="Username"]')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")
    
    # get password tab
    try:
        provided_pass = "Muhssan7865"
        password = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'.//input[@placeholder="Password"]')))
        driver.execute_script("arguments[0].scrollIntoView();", password)
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}") 
    
    # get the check done
    try:
        check = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/input[@value="Login"]')))
        if check:
            driver.execute_script("arguments[0].scrollIntoView();", check)
            check.click()
        
    except Exception as e:
        print(f"No check tab found and the error is {e}") 


    
    results = []
    # while True:
    for i in range(5):
        details= {}
        # car name--> done
        try:
            car_name = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="sc-jKVCRD fRxzLv"]')))
            if car_name:
                details['Title'] = car_name.text.strip()
            else:
                details['Title'] = "na"
        except Exception as e:
            print("No car name")
        
        
        
        # image card --> done   
        try:
            img_clk = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, './/div[contains(@class, "gallery__image")]')))
            img_clk.click()
            time.sleep(2)  # Small delay to allow modal to load

            imgs = []
            while True:
                try:
                    img_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[contains(@class, "modal-gallery__body")]')))
                    img = img_card.find_element(By.TAG_NAME, "img")
                    imgs.append(img.get_attribute("src"))

                    # Locate and click the 'Next' button
                    next_butn = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, './/button[contains(@class, "hRVyLw")]')))
                    next_butn.click()
                    time.sleep(1)  # Delay to allow image change

                except Exception:
                    print("No more images or error clicking next button.")
                    break  # Exit loop when no next button

            # Close the image modal
            try:
                close = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, './/button[text()="Close"]')))
                time.sleep(1)
                close.click()
            except Exception as e:
                print("Error closing image modal:")

            # Store image links
            details['Images'] = ", ".join(imgs)
        except Exception as e:
            print("Error processing image card:")
            
            
        # bid pricing --> done
        try:
            pr_main_card = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle-banner__purchase-block"]')))
            if pr_main_card:
                    prices = WebDriverWait(pr_main_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="sc-kGXeez bMxRxo"]')))
                    if prices:
                        for price in prices:
                            pr_lbl = price.find_element(By.XPATH, './/label[@class="sc-gzVnrw kiynVB pricing-block__header"]').text.strip()
                            pr_val = price.find_element(By.XPATH, './/p[contains(@class, "sc-kAzzGY")]').text.strip()
                            if pr_lbl and pr_val:
                                details[pr_lbl] = pr_val
        except Exception as e:
            print(f"No pricings and error is {e}")
                
        
        # getting tabs information
        try:
            tabs_clck_card = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="tabbed-view-header-block"]')))
            if tabs_clck_card:
            
                # get the veh details --> done
                try:
                    veh_det_clck = WebDriverWait(tabs_clck_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[text()="Vehicle Details"]')))
                    if veh_det_clck:
                        
                        driver.execute_script("arguments[0].scrollIntoView();", veh_det_clck)
                        veh_det_clck.click()
                        
                        # veh det informatin --> done
                        try:
                            veh_info_card = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle-details-top-block"]')))
                            if veh_info_card:
                                veh_rows = WebDriverWait(veh_info_card,5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="vehicle-details-detail-list__row"]')))
                                if veh_rows:
                                    for veh_row in veh_rows:
                                        veh_lbl = veh_row.find_elements(By.XPATH, './/p[@class="sc-htpNat cNoVxz"]')[0].text.strip()
                                        veh_val = veh_row.find_elements(By.XPATH, './/p[@class="sc-htpNat cNoVxz"]')[1].text.strip()
                                        if veh_lbl and veh_val:
                                            details[veh_lbl] = veh_val
                                        else:
                                            print("No veh label and vale")
                                else:
                                    print("No veh rows")
                            else:
                                print("No veh det card")
                        except Exception as e:
                            print("no veh det found")
                            
                        # specifictaion --> done
                        try:
                            spec_card = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="sc-elJkPf bDeocW"]')))
                            if spec_card:
                                spec_btn = WebDriverWait(spec_card,5).until(EC.presence_of_element_located((By.XPATH, './/button[@class="sc-hmzhuo hOMyGw accordion__header"]')))
                                if spec_btn:
                                    time.sleep(1)
                                    spec_btn.click()
                                    
                                    spec_det_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="accordion__content"]'))) 
                                    if spec_det_card:
                                        spec_rows = WebDriverWait(spec_det_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="technical-spec-detail-list__row"]')))
                                        if spec_rows:
                                            for spec_row in spec_rows:
                                                spec_lbl = spec_row.find_elements(By.XPATH, './/p[@class="sc-htpNat cNoVxz"]')[0].text.strip()
                                                spec_val = spec_row.find_elements(By.XPATH, './/p[@class="sc-htpNat cNoVxz"]')[1].text.strip()
                                                if spec_lbl and spec_val:
                                                    details[spec_lbl] = spec_val
                                                else:
                                                    print("No spec label and vale")
                                        else:
                                            print("No spec rows")
                                    else:
                                        print("No spec card")
                                else:
                                    print("No spec button")
                            else:
                                print("No spec click")
                        except Exception as e:
                            print(" No spec card found")
                    else:
                        print("No veh det card to click")
                except Exception as e:
                    print("No veh card")
                
                    
                # Standard Equipment --> done
                try:
                    stnd_det_clck = WebDriverWait(tabs_clck_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[text()="Standard Equipment"]')))
                    if stnd_det_clck:
                        driver.execute_script("arguments[0].scrollIntoView();", stnd_det_clck)
                        stnd_det_clck.click()

                        stnd_info_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="tabbed-view-panel-block"]')))
                        if stnd_info_card:
                            stnd_rows = WebDriverWait(stnd_info_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="text-list__row"]')))
                            if stnd_rows:
                                for stnd_row in stnd_rows:
                                    stnd_lbl = stnd_row.find_element(By.XPATH, './/p[@class="sc-htpNat cNoVxz"]').text.strip()
                                    stnd_vals = stnd_row.find_elements(By.TAG_NAME, 'li')
                                    if stnd_vals:
                                        stnd_val_lst = [li.text.strip() for li in stnd_vals]
                                        if stnd_lbl and stnd_val_lst:
                                            details[stnd_lbl] = stnd_val_lst
                                        else:
                                            print("No standard labels and values")
                                            
                            else:
                                print("No standard rows")
                        else:
                            print("No standard card")
                    else:
                        print("No veh det card to click")
                except Exception as e:
                    print(f"No standard card")
                
                # Inspection Report --> done
                try:
                    inspec_det_clck = WebDriverWait(tabs_clck_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[text()="Inspection Report"]')))
                    if inspec_det_clck:
                        driver.execute_script("arguments[0].scrollIntoView();", inspec_det_clck)
                        inspec_det_clck.click()

                        inspec_card =  WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="tabbed-view-panel-block"]')))
                        if inspec_card:
                            inspec_link = WebDriverWait(inspec_card, 5).until(EC.presence_of_element_located((By.XPATH, './/a[text()="Download PDF"]')))
                            if inspec_link:
                                inspec = inspec_link.get_attribute("href")
                                details['Inspection_report'] = inspec
                            else:
                                details['Inspection_report'] = "na"
                        else:
                            print("No inspec card")
                    else:
                        print("No inspec card to click")
                except Exception as e:
                    print(f"No inspec card")
                    
                # Other Documents --> done
                try:
                    other_det_clck = WebDriverWait(tabs_clck_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[text()="Other Documents"]')))
                    if other_det_clck:
                        driver.execute_script("arguments[0].scrollIntoView();", other_det_clck)
                        other_det_clck.click()

                        other_card =  WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="tabbed-view-panel-block"]')))
                        if other_card:
                            other_link = WebDriverWait(other_card, 5).until(EC.presence_of_element_located((By.XPATH, './/a[@class="downloadBtn gaEvent-download"]')))
                            if other_link:
                                other = other_link.get_attribute("href")
                                details['Other Documents'] = other
                            else:
                                details['Other Documents'] = "na"
                        else:
                            print("No Other Documents card")
                    else:
                        print("No Other Documents to click")
                except Exception as e:
                    print(f"No Other Documents")
                
        except Exception as e:
            print("No tabs to click")
        
        results.append(details)
        
        # next button
        try:
            next_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="sc-bnXvFD kEdhnn"]'))) 
            next_btn =WebDriverWait(next_card, 10).until(EC.presence_of_element_located((By.XPATH, './/span[text()="Next"]'))) 
            if next_btn:
                driver.execute_script("arguments[0].scrollIntoView();", next_btn)
                time.sleep(2)
                next_btn.click()
            else:
                print("No next button link")
                break
        except Exception as e:
            print("No next button link")  
            
    
    
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("1link.csv", index=False)
    driver.quit()
path = "https://disposalnetwork.1link.co.uk/uk/tb/app/vehicle/26969168"
scrape(path)

Error processing image card:
No Other Documents
No car name
No more images or error clicking next button.
No Other Documents
Error processing image card:
No Other Documents
No car name
No more images or error clicking next button.
No Other Documents
Error processing image card:
No Other Documents


In [20]:
df =pd.read_csv("1link.csv")
df

,Title,STARTING BID,BUY NOW,Registration,Grade,Condition,Fuel,Transmission,Registered,Odometer,...,Entertainment,Exterior Body Features,Interior Features,Load Area,Safety,Seats,Security,Wheels,Inspection_report,Images
0,LCV FORD TRANSIT CONNECT\n200 L1 PETROL 1.0 Ec...,"£7,800","£8,200",FH69VDM,-,Clean,Petrol,-,22/01/2020,"10,752 mi",...,['Steering wheel mounted audio controls'],['Active grill shutter'],"['Overhead stowage', 'Reach and rake adjustabl...",['Cargo area tie down loops'],"['Driver seatbelt reminder', 'Drivers airbag']",['4 way adjustable passenger seat'],"['Immobiliser', 'Remote central double locking']","['16"" steel wheels']",https://disposalnetwork.1link.co.uk/,NaN
1,NaN,NaN,"£33,500",YJ73REL,-,Clean,Diesel,-,31/01/2024,"7,385 mi (28/01/2025)",...,['Steering wheel mounted audio controls'],['Active grill shutter'],"['Overhead stowage', 'Reach and rake adjustabl...",['Cargo area tie down loops'],"['Driver seatbelt reminder', 'Drivers airbag']",['4 way adjustable passenger seat'],"['Immobiliser', 'Remote central double locking']","['16"" steel wheels']",https://disposalnetwork.1link.co.uk/,https://disposalnetwork.1link.co.uk/uk/micro/a...
2,SKODA KODIAQ\nESTATE 1.5 TSI SE L Executive 5d...,NaN,"£28,200",YE24RVZ,-,Clean,Petrol,-,31/05/2024,"5,052 mi (30/01/2025)",...,['Steering wheel mounted audio controls'],['Active grill shutter'],"['Overhead stowage', 'Reach and rake adjustabl...",['Cargo area tie down loops'],"['Driver seatbelt reminder', 'Drivers airbag']",['4 way adjustable passenger seat'],"['Immobiliser', 'Remote central double locking']","['16"" steel wheels']",https://disposalnetwork.1link.co.uk/,NaN
3,NaN,NaN,"£28,200",YG24RWL,-,Clean,Petrol,-,30/04/2024,"5,244 mi (30/01/2025)",...,['Steering wheel mounted audio controls'],['Active grill shutter'],"['Overhead stowage', 'Reach and rake adjustabl...",['Cargo area tie down loops'],"['Driver seatbelt reminder', 'Drivers airbag']",['4 way adjustable passenger seat'],"['Immobiliser', 'Remote central double locking']","['16"" steel wheels']",https://disposalnetwork.1link.co.uk/,https://disposalnetwork.1link.co.uk/uk/micro/a...
4,SKODA KODIAQ\nESTATE 1.5 TSI SE L Executive 5d...,NaN,"£28,100",YE24RVF,-,Clean,Petrol,-,31/05/2024,"5,885 mi (30/01/2025)",...,['Steering wheel mounted audio controls'],['Active grill shutter'],"['Overhead stowage', 'Reach and rake adjustabl...",['Cargo area tie down loops'],"['Driver seatbelt reminder', 'Drivers airbag']",['4 way adjustable passenger seat'],"['Immobiliser', 'Remote central double locking']","['16"" steel wheels']",https://disposalnetwork.1link.co.uk/,NaN


In [22]:
df['Images'][1]

'https://disposalnetwork.1link.co.uk/uk/micro/artifacts/vehicleimage/33363183?key=CsCpGKloQNm1iX2qPzuUR5%2FHTEhS7hjRmaQrqjNmMYrhBeaCC%2BYpaZ1DYtGAQxNqA6RVB0hmzjCNo7DAhlxjWA%3D%3D, https://disposalnetwork.1link.co.uk/uk/micro/artifacts/vehicleimage/33363175?key=eYyqJgVCg%2BTz8Q828JI00fCJNrNwl4ZO8dPB3AMLtmyhU8NFWEsz4ouAOPzqLiOFx2rTVFJfJboHULvL7VGkIg%3D%3D, https://disposalnetwork.1link.co.uk/uk/micro/artifacts/vehicleimage/33363176?key=BQ%2B49mn8gP1h5eY7rFLItf4JQ%2BVfUO3tZeDqSl%2B5yloMFW1zSlh%2F%2BsfxqD3O7IQuRt6gpBieAm57IEhrHsLQVQ%3D%3D, https://disposalnetwork.1link.co.uk/uk/micro/artifacts/vehicleimage/33363177?key=%2FLwMRmILL1tm4qWBfeEHEYRvemei%2BupWV%2BK%2BeumFgfkH%2F15l3O%2BvX4RsG1eciZ%2BQq51zdvxypLSUnhLClxn3qg%3D%3D, https://disposalnetwork.1link.co.uk/uk/micro/artifacts/vehicleimage/33363180?key=vUWVGHKefxnaMsIgU5ID6MZzINgA%2BfJZsqtc%2BjxK6kAskTvx5wkucVm0giTBr8ac15zyIF2wb1JJ1yMBA4VMzg%3D%3D, https://disposalnetwork.1link.co.uk/uk/micro/artifacts/vehicleimage/33363181?key=GLsivqZdf

In [23]:
reg_img = df[['Registration', 'Images']]

In [25]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

def download_images(data, main_folder="Images"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Registration"] # separate reg nums
        
        if pd.isna(row["Images"]) or not str(row["Images"]).strip():
            print(f"Skipping {reg_no} due to missing images.")
            continue  # Skip this row
        
        image_urls = row["Images"].split(", ")  # Split image URLs by comma if multiple
        
        # Create a subfolder for the car registration number
        reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
        os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
        
        # loop around the urls of the current row and also save index for further use
        for idx, url in enumerate(image_urls):
            url = url.strip()  # Remove extra spaces
            if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                url = urljoin("https://", url) # sets the urls starting from https://.....
            
            # parse the url
            parsed_url = urlparse(url)

            # check if the parsed url is valid
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
                continue
            
            # try downloading images
            try:
                # Download the image
                response = requests.get(url, stream=True) # send the url for downloading
                response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                
                # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                # If no file name is found, assigns a default name based on the index.
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                # Extracts the file extension (e.g., jpg). from the last index value
                file_extension = file_name.split(".")[-1]
                
                # Ensure the file has a valid extension
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                
                # Construct the full path for the file
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): # Reads the response in 1KB chunks to save memory.
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")
# Call the function
download_images(reg_img)


Skipping FH69VDM due to missing images.
Downloaded: Images\YJ73REL\YJ73REL_1_image_1.jpg
Downloaded: Images\YJ73REL\YJ73REL_2_image_2.jpg
Downloaded: Images\YJ73REL\YJ73REL_3_image_3.jpg
Downloaded: Images\YJ73REL\YJ73REL_4_image_4.jpg
Downloaded: Images\YJ73REL\YJ73REL_5_image_5.jpg
Downloaded: Images\YJ73REL\YJ73REL_6_image_6.jpg
Downloaded: Images\YJ73REL\YJ73REL_7_image_7.jpg
Downloaded: Images\YJ73REL\YJ73REL_8_image_8.jpg
Downloaded: Images\YJ73REL\YJ73REL_9_image_9.jpg
Downloaded: Images\YJ73REL\YJ73REL_10_image_10.jpg
Downloaded: Images\YJ73REL\YJ73REL_11_image_11.jpg
Downloaded: Images\YJ73REL\YJ73REL_12_image_12.jpg
Downloaded: Images\YJ73REL\YJ73REL_13_image_13.jpg
Downloaded: Images\YJ73REL\YJ73REL_14_image_14.jpg
Skipping YE24RVZ due to missing images.
Downloaded: Images\YG24RWL\YG24RWL_1_image_1.jpg
Downloaded: Images\YG24RWL\YG24RWL_2_image_2.jpg
Downloaded: Images\YG24RWL\YG24RWL_3_image_3.jpg
Downloaded: Images\YG24RWL\YG24RWL_4_image_4.jpg
Downloaded: Images\YG24RWL\Y